In [24]:
# -*- coding: UTF-8 -*-
import numpy as np
import pandas as pd
import jieba
import jieba.analyse
import codecs
import openpyxl as op
import re
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

%config IPCompleter.greedy=True

seg1 = jieba.cut("好好学学python，有用。", cut_all=True)
print('全模式：', ' '.join(seg1))
seg2 = jieba.cut("好好学学python，有用。", cut_all=False)
print("精确模式（也是默认模式）：", ' '.join(seg2))
seg3 = jieba.cut_for_search("好好学学python，有用。")
print("搜索引擎模式：", ' '.join(seg3))

全模式： 好好 好好学 好学 学学 python  有用  
精确模式（也是默认模式）： 好好 学学 python ， 有用 。
搜索引擎模式： 好好 学学 python ， 有用 。


In [2]:
# df=pd.read_excel('C:\\Users\\user\\Desktop\\大三上\\机器学习\\程序\\大作业\\rowdate.xlsx')
# 打开excel文件,获取工作簿对象
wb = op.load_workbook('rowdate.xlsx')
ws = wb.active

rows=[]
for row in ws.iter_rows():
    rows.append(row)
rows[2][2].value


cols=[]
cols = []
for col in ws.iter_cols():
    cols.append(col)
 
cols   #所有列
cols[0]   #获取第一列
cols[0][0]   #获取第一列的第一行的单元格对象
cols[0][0].value   #获取第一列的第一行的值

'Id'

In [3]:
disTitle=""          #所有数据的标题
disDis=""            #所有数据的疾病
disDescr=""          #所有数据的病情描述
disTotal=""          #所有数据的标题+疾病+病情描述
personDate=[]        #个人数据组成的列表
personType=[]        #个人数据的分类
disCode={'胃炎':0,
     '胃癌':1,
     '肠胃炎':2,
     '慢性胃炎':3,
     '慢性前变性胃炎':4}

for i in range(1,len(cols[5])):
    person=""
    if cols[1][i].value != None:
        disTitle += cols[1][i].value
        disTotal += cols[1][i].value
        person+=cols[1][i].value
        
    if cols[2][i].value != None:
        disDis += cols[2][i].value
        disTotal += cols[2][i].value
        person+=cols[2][i].value
        
    if cols[3][i].value != None:
        disDescr+= cols[3][i].value
        disTotal+= cols[3][i].value
        person+=cols[3][i].value
#         print(person)
    personDate.append(person)
    personType.append(disCode[cols[4][i].value])
# print(personDate)
# print('\n'*40)
# print(disDis)
# print('\n'*40)
# print(disDescr)
print(personDate[0])

 胆汁反流性胃炎，胸口堵的疼，嗳气，呕吐怎么办？胆汁反流性胃炎，胸口堵的厉害，嗳气。倪主任您好，我今年三月查出胆汁反流，吃了两个多月中药和奥美拉错，莫沙必利，没有多大效果，之后改服铝碳酸镁片跟莫沙必利，现在感觉还是没有什么作用，主要症状就是整个食道到嗓子都堵着有灼烧感，堵的厉害整个胸前都疼，经常嗳气，前天夜里胃突然开始疼，之后开始吐，吐出来后疼痛才减轻一点，我的胆汁反流的症状是不是越来越严重了？


In [33]:
# 创建停用词list
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

# 对句子进行分词
def seg_sentence(sentence):
    jieba.load_userdict('./dict.txt')
#     jieba.analyse.set_stop_words("中文停用词表.txt")
#     jieba.analyse.set_idf_path("../extra_dict/idf.txt.big");
    sentence_seged = jieba.cut(sentence.strip())
    stopwords = stopwordslist('./哈工大停用词表.txt')  # 这里加载停用词的路径
    outstr = []
    for word in sentence_seged:
        if word not in stopwords:
            if re.match(r'[\u4e00-\u9fa5][\u4e00-\u9fa5]+$',word,flags=0) and not re.match(r'(一)[\u4e00-\u9fa5]+$',word,flags=0):
                outstr.append(word)
    return outstr

In [34]:
# 对‘标题’列分词
segTitle = seg_sentence(disTitle)
# print(segTitle)
Counter(segTitle).most_common(50)

[('肠胃炎', 534),
 ('胃炎', 465),
 ('慢性胃炎', 342),
 ('胃癌', 318),
 ('慢性浅表性胃炎', 300),
 ('萎缩性', 234),
 ('慢性', 194),
 ('急性', 175),
 ('糜烂', 135),
 ('咨询', 74),
 ('患者', 68),
 ('胆汁反流', 67),
 ('浅表性', 60),
 ('胃镜', 55),
 ('胃窦', 52),
 ('术后', 52),
 ('诊后', 48),
 ('十二指肠', 47),
 ('转移', 47),
 ('食管炎', 43),
 ('手术', 41),
 ('息肉', 38),
 ('晚期', 38),
 ('溃疡', 35),
 ('是不是', 35),
 ('胆汁', 30),
 ('幽门螺杆菌', 30),
 ('反流性', 27),
 ('发烧', 27),
 ('是否', 25),
 ('糜烂性', 24),
 ('腹泻', 24),
 ('呕吐', 23),
 ('返流', 22),
 ('贲门', 22),
 ('诊断', 21),
 ('食管', 21),
 ('轻度', 20),
 ('反复', 20),
 ('中度', 20),
 ('化疗', 20),
 ('拉肚子', 19),
 ('消化不良', 18),
 ('幽门', 18),
 ('方案', 18),
 ('宝宝', 18),
 ('症状', 17),
 ('严重', 17),
 ('引起', 17),
 ('建议', 17)]

In [35]:
# 对‘疾病’列分词
segDis = seg_sentence(disDis)
count = Counter(segDis)
count.most_common(50)

[('胃炎', 443),
 ('肠胃炎', 270),
 ('慢性浅表性胃炎', 243),
 ('慢性胃炎', 218),
 ('萎缩性', 205),
 ('胃癌', 202),
 ('慢性', 191),
 ('糜烂', 138),
 ('急性', 119),
 ('胆汁反流', 70),
 ('浅表性', 68),
 ('十二指肠', 56),
 ('胃窦', 53),
 ('食管炎', 44),
 ('呕吐', 41),
 ('息肉', 39),
 ('腹泻', 39),
 ('溃疡', 37),
 ('胃病', 35),
 ('幽门螺杆菌', 33),
 ('拉肚子', 33),
 ('转移', 33),
 ('中度', 32),
 ('肚子', 32),
 ('胆汁', 31),
 ('发烧', 31),
 ('返流', 30),
 ('轻度', 29),
 ('糜烂性', 28),
 ('大便', 28),
 ('术后', 28),
 ('肠胃', 27),
 ('胃镜', 26),
 ('结肠炎', 25),
 ('反流性', 24),
 ('食管', 24),
 ('阳性', 23),
 ('胃部', 22),
 ('腹部', 22),
 ('恶心', 22),
 ('晚期', 22),
 ('增生', 21),
 ('幽门', 21),
 ('手术', 21),
 ('胃疼', 20),
 ('球部', 20),
 ('胃溃疡', 20),
 ('肠炎', 19),
 ('头晕', 19),
 ('胃胀', 18)]

In [37]:
# 对‘病情描述’列分词
segDescr = seg_sentence(disDescr)
count = Counter(segDescr)
count.most_common(50)

[('胃镜', 566),
 ('胃炎', 361),
 ('大便', 359),
 ('肚子', 250),
 ('症状', 246),
 ('难受', 237),
 ('舒服', 232),
 ('吃饭', 205),
 ('胃癌', 174),
 ('呕吐', 169),
 ('正常', 167),
 ('打嗝', 164),
 ('左右', 163),
 ('拉肚子', 162),
 ('胃胀', 160),
 ('腹部', 158),
 ('手术', 156),
 ('萎缩性', 152),
 ('肠胃炎', 148),
 ('严重', 140),
 ('慢性', 140),
 ('情况', 139),
 ('疼痛', 138),
 ('好转', 138),
 ('今天', 138),
 ('腹泻', 133),
 ('中药', 130),
 ('胃部', 130),
 ('不好', 129),
 ('恶心', 128),
 ('嗳气', 125),
 ('效果', 123),
 ('诊断', 123),
 ('服用', 121),
 ('伤害', 120),
 ('出现', 119),
 ('化疗', 112),
 ('慢性浅表性胃炎', 110),
 ('发烧', 103),
 ('糜烂', 100),
 ('几天', 100),
 ('已经', 100),
 ('问题', 99),
 ('胃痛', 98),
 ('知道', 97),
 ('之前', 96),
 ('之后', 95),
 ('有时候', 94),
 ('慢性胃炎', 93),
 ('后来', 92)]

In [8]:
# segDescr = seg_sentence(''.join(segDescr))
# count = Counter(segDescr)
# count.most_common(50)
# jieba.suggest_freq('慢性浅表性胃炎',True)
# jieba.suggest_freq('慢性胃炎',True)
# 病情描述列计算df-idf值并列出前50个
key=jieba.analyse.extract_tags(''.join(segDescr),topK=50, withWeight=True)
print(key)

[('胃镜', 0.15775709129923643), ('胃炎', 0.09323850021480073), ('大便', 0.07826136009391423), ('感觉', 0.06878129198112283), ('医生', 0.06534525517215978), ('检查', 0.061269229108732226), ('医院', 0.05650867203206864), ('胃胀', 0.04758682237535299), ('难受', 0.045579178115033044), ('肠胃炎', 0.04554134491903665), ('打嗝', 0.045440219970348486), ('胃癌', 0.04482320735625376), ('肚子', 0.0442142995191453), ('治疗', 0.04374479114315041), ('舒服', 0.04373974031751452), ('萎缩性', 0.0431361581404967), ('拉肚子', 0.04284536561333116), ('症状', 0.04027658677847086), ('幽门', 0.03967455388707691), ('现在', 0.036821402099819746), ('嗳气', 0.03482403154948177), ('没有', 0.033665626380552774), ('胃部', 0.033490373808181456), ('呕吐', 0.03330428538832616), ('慢性浅表性胃炎', 0.033220989205435106), ('吃饭', 0.03307294549479271), ('有点', 0.03083443342083918), ('晚上', 0.030599278506560433), ('腹部', 0.02958630043052273), ('请问', 0.02899950886758587), ('有时', 0.028497277793200982), ('腹泻', 0.028402144751953986), ('化疗', 0.028064090317364312), ('早上', 0.0279085401652368

In [62]:
# 对个人数据进行单独分词
segPersonDate=[]
for i in range(0,len(personDate)):
    segPerson = seg_sentence(personDate[i])
    segPersonDate.append(' '.join(segPerson))
print(segPersonDate[0])

胆汁反流 胃炎 胸口 嗳气 呕吐 胆汁反流 胃炎 胸口 厉害 嗳气 您好 今年 三月 查出 胆汁反流 两个 多月 中药 奥美拉错 莫沙必利 多大 效果 之后 改服 铝碳酸镁片 莫沙必利 作用 主要症状 整个 食道 嗓子 灼烧 厉害 整个 胸前 嗳气 前天 夜里 突然 之后 吐出来 疼痛 减轻 胆汁反流 症状 是不是 越来越 严重


In [41]:
# X_test = [' 胆汁反流性胃炎，胸口堵的疼，嗳气，呕吐怎么办','没有 你 的 旅行 都是 流浪']
# #但可以设token_pattern=r"(?u)\b\w+\b"，即不考虑停用词
test=[]
test.append(segPersonDate[0])
test.append(segPersonDate[1])
test.append(segPersonDate[2])
test.append(segPersonDate[3])
print(test)

['胆汁反流 胃炎 胸口 嗳气 呕吐 胆汁反流 胃炎 胸口 厉害 嗳气 您好 今年 三月 查出 胆汁反流 两个 多月 中药 奥美拉错 莫沙必利 多大 效果 之后 改服 铝碳酸镁片 莫沙必利 作用 主要症状 整个 食道 嗓子 灼烧 厉害 整个 胸前 嗳气 前天 夜里 突然 之后 吐出来 疼痛 减轻 胆汁反流 症状 是不是 越来越 严重', '慢性胃炎 嗳气 胃疼 嗳气 前查 幽门螺杆菌 后来 阿奇 霉素 阿莫西林 尿囊素 雷贝拉唑 胃疼 之前 还好 知道 是不是 刺激 胃粘膜 不是 床上 白天 还好 嗳气 胃病 小学 大学 雷贝拉唑 第五 效果 不是 特别 往年 中药 短时间 网上 预约 不到 周五 学校 请假 不能 加个', '胃窦 慢性 萎缩性 胃炎 胃窦 慢性 萎缩性 胃炎 胃痛 保定 五二 胃镜 病理 诊断 胃窦 慢性 萎缩性 胃炎 伴肠化 上皮 轻度 典型 增生 幽门螺杆菌 测试 测试 复查 幽门螺杆菌 测试', '混合 肠胃炎 咨询 混合 李医生 您好 月底 肠胃炎 大便 后来 发现 痔疮 脱出 越来越 鲜血 看后 混合 里面 糜烂 建议 痔疮 多年 比较严重']


In [67]:
# vectorizer = TfidfVectorizer(stop_words=stopwordslist('./tfidfstop.txt'), min_df=20)
vectorizer = TfidfVectorizer(min_df=5)
result=vectorizer.fit_transform(segPersonDate)
print(len(vectorizer.get_feature_names()))
print(vectorizer.get_feature_names())

matrix = vectorizer.transform(segPersonDate)    # transform()会把每篇文档转换为向量；得到tf-idf矩阵；
print(matrix.shape)

1263
['丁啉', '七天', '三个', '三九', '三周', '三四个', '三四次', '三天', '三年', '三月', '三次', '三点', '三种', '三联', '上传', '上午', '上吐下泻', '上周', '上方', '上来', '上次', '上海', '上班', '上皮', '上腹', '上述', '上面', '下午', '下半年', '下去', '下咽', '下方', '下泄', '下腹', '下腹部', '下部', '下降', '下面', '不久', '不了', '不会', '不佳', '不停', '不再', '不准', '不到', '不吃', '不吐', '不大', '不太', '不太好', '不好', '不少', '不尽', '不得', '不怎么', '不想', '不成形', '不敢', '不断', '不是', '不止', '不用', '不畅', '不疼', '不痛', '不知', '不能', '不行', '不要', '不见', '不足', '不适', '不适感', '专家', '两三天', '两三次', '两个', '两周', '两天', '两年', '两次', '两点', '两边', '严重', '个人', '中医', '中医院', '中午', '中山', '中度', '中心', '中心医院', '中性', '中成药', '中旬', '中晚期', '中期', '中药', '中西药', '中间', '临时', '为主', '主要', '主要症状', '丽珠', '久治不愈', '之前', '之后', '之间', '乏力', '乙状结肠', '乙肝', '九月份', '九点', '习惯', '乳酸菌', '了解', '二个月', '二十多年', '二十年', '二年', '二次', '五六次', '五天', '五年', '五次', '五点', '交替', '人体', '人民', '今天', '今年', '介入', '仍然', '从来', '以上', '以为', '以前', '以后', '以来', '休息', '优先', '会痛', '伤害', '伴有', '伴肠', '伴肠化', '伴随', '位置', '低烧', '住院', '住院治疗', '体检', '体温', '体质', '体重', '作用', '你好', '使用', 

In [68]:
# 利用逻辑回归模型对数据进行预测
logistic_model = LogisticRegression(multi_class = 'multinomial', solver='lbfgs')
sum = 0;
for i in range(0,10):
    train_X, test_X, train_y, test_y = train_test_split(matrix, personType, test_size=0.2)
    logistic_model.fit(train_X, train_y)
    score = logistic_model.score(test_X, test_y)
    print("第", i, "次迭代的分数是：" , score)
    sum += score
print("逻辑回归平均分数为：" , sum/10)
print(test_y[:10])
print(logistic_model.predict(test_X)[:10])

第 0 次迭代的分数是： 0.8311688311688312
第 1 次迭代的分数是： 0.8311688311688312
第 2 次迭代的分数是： 0.7974025974025974
第 3 次迭代的分数是： 0.8233766233766234
第 4 次迭代的分数是： 0.8623376623376623
第 5 次迭代的分数是： 0.8025974025974026
第 6 次迭代的分数是： 0.8649350649350649
第 7 次迭代的分数是： 0.8207792207792208
第 8 次迭代的分数是： 0.7974025974025974
第 9 次迭代的分数是： 0.8181818181818182
逻辑回归平均分数为： 0.824935064935065
[4, 4, 2, 2, 2, 0, 1, 4, 0, 4]
[4 0 2 2 2 0 1 4 4 0]


In [60]:
decision_tree_model = tree.DecisionTreeClassifier()
sum = 0;
for i in range(0,10):
    train_X, test_X, train_y, test_y = train_test_split(matrix, personType, test_size=0.2)
    decision_tree_model.fit(train_X, train_y)
    score = decision_tree_model.score(test_X, test_y)
    print("第", i, "次迭代的分数是：" , score)
    sum += score
print("决策树平均分数为：" , sum/10)
print(test_y[:10])
print(decision_tree_model.predict(test_X)[:10])

第 0 次迭代的分数是： 0.8415584415584415
第 1 次迭代的分数是： 0.7896103896103897
第 2 次迭代的分数是： 0.8415584415584415
第 3 次迭代的分数是： 0.8051948051948052
第 4 次迭代的分数是： 0.8389610389610389
第 5 次迭代的分数是： 0.8207792207792208
第 6 次迭代的分数是： 0.8103896103896104
第 7 次迭代的分数是： 0.8519480519480519
第 8 次迭代的分数是： 0.812987012987013
第 9 次迭代的分数是： 0.7948051948051948
决策树平均分数为： 0.8207792207792208
[2, 0, 2, 0, 3, 4, 3, 1, 4, 3]
[2 2 0 2 0 4 3 1 4 3]


In [61]:
# 利用SVM对模型进行训练和预测
svm_model = svm.SVC(C=2, kernel='linear', decision_function_shape='ovo')
sum = 0;
for i in range(0,10):
    train_X, test_X, train_y, test_y = train_test_split(matrix, personType, test_size=0.2)
    svm_model.fit(train_X, train_y)
    score = svm_model.score(test_X, test_y)
    print("第", i, "次迭代的分数是：" , score)
    sum += score
print("SVM平均分数为：" , sum/10)
print(test_y[:10])
print(svm_model.predict(test_X)[:10])

第 0 次迭代的分数是： 0.8
第 1 次迭代的分数是： 0.7974025974025974
第 2 次迭代的分数是： 0.8285714285714286
第 3 次迭代的分数是： 0.825974025974026
第 4 次迭代的分数是： 0.8441558441558441
第 5 次迭代的分数是： 0.8519480519480519
第 6 次迭代的分数是： 0.8207792207792208
第 7 次迭代的分数是： 0.825974025974026
第 8 次迭代的分数是： 0.8467532467532467
第 9 次迭代的分数是： 0.8025974025974026
SVM平均分数为： 0.8244155844155845
[1, 1, 0, 2, 1, 3, 1, 0, 0, 4]
[1 1 0 2 1 3 1 4 0 4]


In [26]:
# 利用SVM对模型进行训练和预测
bys_model = GaussianNB()
sum = 0;
for i in range(0,10):
    train_X, test_X, train_y, test_y = train_test_split(matrix, personType, test_size=0.2)
    bys_model.fit(train_X), train_y)
    score = bys_model.score(test_X, test_y)
    print("第", i, "次迭代的分数是：" , score)
    sum += score
print("SVM平均分数为：" , sum/10)
print(test_y[:10])
print(bys_model.predict(test_X)[:10])

AttributeError: to_array not found